In [ ]:
import datetime
import warnings
import time
import numpy as np
import pandas as pd
from pmdarima.arima import auto_arima

warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

temp = trans[trans.columns[1:5]].drop_duplicates()
new_trans = pd.DataFrame()
for dt in trans['date_dt'].drop_duplicates().values:
    temp['date_dt'] = dt
    new_trans = pd.concat([new_trans,temp],axis=0,ignore_index=True)
new_trans = pd.merge(new_trans,trans,how='left',on=new_trans.columns.tolist()).fillna(0)
new_trans = new_trans[trans.columns]
trans = new_trans.sort_values(by=['date_dt'])
del new_trans

date_dt = []
for i in range(15):
    date_dt.append(20180302+i)
cols = ['o_city_code', 'o_district_code', 'd_city_code',
   'd_district_code']

district_code_values = trans[cols].drop_duplicates().values
preds_df = pd.DataFrame()
num = 0
s = time.time()
for va in district_code_values[:3000]:
    num+=1
    if num%100==0:
        print(num,time.time()-s,'s')
        s = time.time()
        print('\n')
        print('\n')
        print('\n')
    sub_df = trans[(trans[cols[0]]==va[0])&(trans[cols[1]]==va[1])&(trans[cols[2]]==va[2])&(trans[cols[3]]==va[3])]

    temp_df = pd.DataFrame(data=date_dt, columns=['date_dt'])
    for i in range(4):
        temp_df[cols[i]] = va[i]

    column = 'cnt'
    ts_log = np.log(1 + sub_df[column])
    arima_model = auto_arima(ts_log, start_p=1, max_p=9, start_q=1, max_q=9, max_d=5,
                             start_P=1, max_P=9, start_Q=1, max_Q=9, max_D=5,
                             m=7, random_state=2018,
                             trace=True,
                             seasonal=True,
                             error_action='ignore',
                             suppress_warnings=True,
                             stepwise=True)

    preds = arima_model.predict(n_periods=15)
    preds = pd.Series(preds)
    preds = np.exp(preds) - 1
    temp_df['cnt'] = preds
    preds_df = pd.concat([preds_df, temp_df], axis=0, ignore_index=True)
sub  = preds_df.sort_values(by=['date_dt'])

res = pd.read_csv('prediction1.csv',header=None)
res.columns = flow.columns.tolist()

temp = res[['date_dt','district_code']]
cols = ['date_dt','o_district_code']
temp.columns = cols
res['flow_out'] = pd.merge(temp,sub.groupby(cols)['cnt'].sum().reset_index(),how='left',on=cols)['cnt']


temp = res[['date_dt','district_code']]
cols = ['date_dt','d_district_code']
temp.columns = cols
res['flow_in'] = pd.merge(temp,sub.groupby(cols)['cnt'].sum().reset_index(),how='left',on=cols)['cnt']
res.to_csv('result/predicition1_trans.csv',header=None,index=False)
1293